# Statistics with Python — One command per cell with comments and examples
#
# This notebook-style script collects essential statistical commands for data science in Python.
#
# Rules followed:
# - Exactly one executable command per code cell (lines beginning with `#%%`).
# - Each cell includes comments explaining what the command does and a minimal example.
# - We use standard libraries: numpy, pandas, scipy.stats, statsmodels, seaborn/matplotlib where helpful.
# - Random seeds are set where randomness is involved to make examples reproducible.


# Setup: import core libraries
# - numpy: numerical computing
# - pandas: tabular data handling
# - scipy.stats: distributions and statistical tests
# - statsmodels: additional statistical utilities (confidence intervals, regression)
# - seaborn/matplotlib: quick statistical visualizations

In [ ]:
import numpy as np  # numerical arrays and math


In [ ]:
import pandas as pd  # DataFrame/Series containers


In [ ]:
from scipy import stats  # distributions and hypothesis testing


In [ ]:
import statsmodels.api as sm  # regression and qq plots


In [ ]:
import statsmodels.stats.api as sms  # confidence intervals, effect sizes, tests


In [ ]:
from statsmodels.stats import proportion as sm_prop  # proportion-specific utilities


In [ ]:
import matplotlib.pyplot as plt  # plotting


In [ ]:
import seaborn as sns  # statistical plots


# Setup: create a small synthetic dataset for examples
# - We'll simulate a normal variable `x` and a slightly shifted `y` for A/B comparisons.
# - Also create a categorical variable and a binary outcome for proportion/chi-square examples.

In [ ]:
np.random.seed(42)  # ensure reproducibility for random examples


In [ ]:
x = np.random.normal(loc=0.0, scale=1.0, size=200)  # sample from N(0, 1)


In [ ]:
y = np.random.normal(loc=0.3, scale=1.1, size=200)  # sample from N(0.3, 1.1)


In [ ]:
cat = np.random.choice(['A', 'B', 'C'], size=200, p=[0.4, 0.4, 0.2])  # categorical labels


In [ ]:
binary = (np.random.rand(200) < 0.42).astype(int)  # Bernoulli variable with p≈0.42


In [ ]:
df = pd.DataFrame({'x': x, 'y': y, 'group': cat, 'clicked': binary})  # assemble DataFrame


# Basic descriptive statistics (numpy/pandas)


In [ ]:
np.mean(x)  # arithmetic mean of array-like data


In [ ]:
np.median(x)  # median (50th percentile)


In [ ]:
stats.mode(cat, keepdims=True)  # mode (most common category); returns value and count


In [ ]:
np.var(x, ddof=0)  # population variance (ddof=0). Use ddof=1 for sample variance


In [ ]:
np.std(x, ddof=0)  # population standard deviation


In [ ]:
pd.Series(x).mad()  # mean absolute deviation (about the mean) for a Series


In [ ]:
np.quantile(x, 0.25)  # 25th percentile (first quartile)


In [ ]:
np.percentile(x, [25, 50, 75])  # percentiles at 25th, 50th, 75th


In [ ]:
np.ptp(x)  # peak-to-peak (max - min), i.e., range of values


In [ ]:
pd.Series(x).skew()  # sample skewness (asymmetry of the distribution)


In [ ]:
pd.Series(x).kurtosis()  # sample excess kurtosis (tailedness) — Fisher definition


In [ ]:
pd.Series(x).describe()  # summary stats: count, mean, std, min, quartiles, max


# Robust statistics


In [ ]:
np.median(np.abs(x - np.median(x)))  # MAD (median absolute deviation) — robust to outliers


In [ ]:
stats.trim_mean(x, proportiontocut=0.1)  # 10% trimmed mean (drops extremes before averaging)


In [ ]:
stats.iqr(x, rng=(25, 75))  # interquartile range (Q3 - Q1)


# Weighted statistics


In [ ]:
np.average(x, weights=np.linspace(1, 2, len(x)))  # weighted mean using increasing weights


In [ ]:
stats.gmean(np.abs(x) + 1e-6)  # geometric mean (positive inputs); small epsilon added


In [ ]:
stats.hmean(np.abs(x) + 1e-6)  # harmonic mean (positive inputs)


# Correlation and covariance


In [ ]:
np.corrcoef(x, y)[0, 1]  # Pearson correlation coefficient between x and y


In [ ]:
pd.Series(x).corr(pd.Series(y), method='pearson')  # Pearson correlation via pandas


In [ ]:
stats.spearmanr(x, y)  # Spearman rank correlation (non-parametric); returns statistic and p-value


In [ ]:
stats.kendalltau(x, y)  # Kendall’s tau correlation (robust to outliers/ordinal data)


In [ ]:
np.cov(x, y, ddof=1)  # sample covariance matrix for x and y


# One-sample and two-sample tests (parametric)


In [ ]:
stats.ttest_1samp(x, popmean=0.0)  # one-sample t-test: H0 mean(x)=0


In [ ]:
stats.ttest_ind(x, y, equal_var=False)  # two-sample Welch’s t-test (unequal variances)


In [ ]:
stats.ttest_rel(x[:100], y[:100])  # paired t-test on first 100 paired observations


In [ ]:
stats.f_oneway(x[cat == 'A'], x[cat == 'B'], x[cat == 'C'])  # one-way ANOVA across groups


# Non-parametric tests (when normality/variance assumptions fail)


In [ ]:
stats.mannwhitneyu(x[cat == 'A'], x[cat == 'B'], alternative='two-sided')  # Mann–Whitney U (independent)


In [ ]:
stats.wilcoxon(x[:100], y[:100])  # Wilcoxon signed-rank test (paired, non-parametric)


In [ ]:
stats.kruskal(x[cat == 'A'], x[cat == 'B'], x[cat == 'C'])  # Kruskal–Wallis test (k-group non-parametric)


# Normality and variance tests


In [ ]:
stats.shapiro(x[:500])  # Shapiro–Wilk normality test (use small N; here N=200 capped by slice)


In [ ]:
stats.normaltest(x)  # D’Agostino and Pearson’s K^2 test for normality


In [ ]:
stats.anderson(x, dist='norm')  # Anderson–Darling test for normality


In [ ]:
stats.levene(x[cat == 'A'], x[cat == 'B'], x[cat == 'C'])  # Levene’s test for equal variances


In [ ]:
stats.bartlett(x[cat == 'A'], x[cat == 'B'], x[cat == 'C'])  # Bartlett’s test for homoscedasticity (normality assumed)


# Categorical data: proportions and contingency tables


In [ ]:
sm_prop.proportions_ztest(count=np.sum(binary), nobs=len(binary), value=0.5)  # one-proportion z-test vs 0.5


In [ ]:
sm_prop.proportion_confint(count=np.sum(binary), nobs=len(binary), method='wilson')  # CI for a single proportion


In [ ]:
ct = pd.crosstab(df['group'], df['clicked'])  # contingency table: rows=group, cols=clicked


In [ ]:
stats.chi2_contingency(ct)  # chi-square test of independence for contingency table


In [ ]:
stats.chisquare(ct.sum(axis=1))  # chi-square goodness-of-fit on category counts (vs. uniform expectation)


# Probability distributions (scipy.stats)


In [ ]:
stats.norm.pdf(0.0, loc=0, scale=1)  # normal PDF at x=0 for N(0,1)


In [ ]:
stats.norm.cdf(1.96)  # normal CDF at x=1.96 (≈0.975)


In [ ]:
stats.norm.ppf(0.975)  # normal quantile for p=0.975 (≈1.96)


In [ ]:
stats.norm.rvs(loc=0, scale=1, size=5, random_state=0)  # sample 5 values from N(0,1)


In [ ]:
stats.t.ppf(0.975, df=20)  # t quantile at 0.975 with 20 degrees of freedom


In [ ]:
stats.chi2.cdf(10, df=5)  # chi-square CDF at x=10, df=5


In [ ]:
stats.f.ppf(0.95, dfn=2, dfd=20)  # F distribution 95th percentile (df1=2, df2=20)


In [ ]:
stats.binom.pmf(k=3, n=10, p=0.4)  # binomial PMF: P(X=3) for X~Binom(n=10,p=0.4)


In [ ]:
stats.poisson.cdf(2, mu=1.5)  # Poisson CDF: P(X<=2) for λ=1.5


In [ ]:
stats.expon.ppf(0.9, scale=2.0)  # exponential quantile at 0.9 with mean=scale=2


In [ ]:
stats.uniform.rvs(loc=-1, scale=2, size=3, random_state=1)  # 3 samples from Uniform(-1, 1)


In [ ]:
stats.bernoulli.rvs(p=0.3, size=5, random_state=2)  # 5 Bernoulli trials with p=0.3


In [ ]:
stats.norm.fit(x)  # MLE fit of normal parameters (mu, sigma) to data x


# Confidence intervals


In [ ]:
stats.t.interval(alpha=0.95, df=len(x)-1, loc=np.mean(x), scale=stats.sem(x))  # 95% CI for mean via t


In [ ]:
sms.DescrStatsW(x).tconfint_mean(alpha=0.05)  # 95% CI for mean using statsmodels helper


In [ ]:
sm_prop.proportion_confint(count=np.sum(binary), nobs=len(binary), method='beta')  # 95% CI for a proportion (Clopper–Pearson)


In [ ]:
sms.CompareMeans(sms.DescrStatsW(x), sms.DescrStatsW(y)).tconfint_diff(usevar='unequal')  # CI for difference in means (Welch)


# Effect sizes


In [ ]:
(np.mean(x) - 0.0) / np.std(x, ddof=1)  # one-sample Cohen’s d vs 0 (standardized mean difference)


In [ ]:
(np.mean(x) - np.mean(y)) / np.sqrt(((np.var(x, ddof=1) + np.var(y, ddof=1)) / 2))  # Cohen’s d for two independent samples


In [ ]:
np.corrcoef(x, y)[0, 1] ** 2  # R-squared (variance explained) for linear relationship between x and y


# Multiple testing correction


In [ ]:
from statsmodels.stats.multitest import multipletests  # adjust p-values for multiple comparisons


In [ ]:
multipletests(pvals=np.random.rand(10), alpha=0.05, method='fdr_bh')  # Benjamini–Hochberg FDR control on 10 p-values


# Linear regression (statsmodels)


In [ ]:
X = sm.add_constant(df[['x']])  # add intercept term to predictor(s)


In [ ]:
ols_model = sm.OLS(df['y'], X).fit()  # fit OLS regression: y ~ 1 + x


In [ ]:
ols_model.params  # regression coefficients (intercept and slope)


In [ ]:
ols_model.summary()  # full regression summary (coefficients, t-stats, R^2, etc.)


# Logistic regression for binary outcomes


In [ ]:
X_log = sm.add_constant(df[['x']])  # design matrix with intercept for logistic regression


In [ ]:
logit_model = sm.Logit(df['clicked'], X_log).fit(disp=False)  # fit logistic regression: logit(click) ~ 1 + x


In [ ]:
logit_model.params  # logistic regression coefficients (log-odds scale)


In [ ]:
logit_model.predict(X_log)[:5]  # predicted click probabilities for first 5 rows


# Resampling methods


In [ ]:
np.random.choice(x, size=len(x), replace=True).mean()  # bootstrap resample mean (one bootstrap sample)


In [ ]:
np.random.permutation(np.concatenate([np.ones(100), np.zeros(100)])).reshape(2, 100).mean(axis=1)  # permutation idea demo


# Time series diagnostics


In [ ]:
import statsmodels.tsa.stattools as tsast  # ACF/PACF and stationarity tests


In [ ]:
tsast.acf(x, fft=True, nlags=20)  # autocorrelation function up to lag 20


In [ ]:
tsast.pacf(x, method='ywm', nlags=20)  # partial autocorrelation up to lag 20


In [ ]:
tsast.adfuller(x)  # Augmented Dickey–Fuller test for unit root (stationarity)


In [ ]:
from statsmodels.tsa.stattools import kpss  # KPSS stationarity test (complements ADF)


In [ ]:
kpss(x, regression='c', nlags='auto')  # KPSS test for level stationarity


# Power and sample size (two-sample t-test example)


In [ ]:
from statsmodels.stats.power import TTestIndPower  # power analysis for independent t-test


In [ ]:
power_analysis = TTestIndPower()  # create power analysis object


In [ ]:
power_analysis.solve_power(effect_size=0.5, power=0.8, alpha=0.05, alternative='two-sided')  # compute required sample size per group


# Outlier detection helpers


In [ ]:
z_scores = stats.zscore(x)  # z-scores: standardize x to mean 0, sd 1 (outlier detection)


In [ ]:
np.where(np.abs(z_scores) > 3)[0]  # indices of potential outliers with |z| > 3


# QQ plots (visual normality check)


In [ ]:
sm.qqplot(pd.Series(x), line='s')  # QQ plot against theoretical normal with standardized line


In [ ]:
plt.show()  # render last plot (required in script-style notebooks)


# Distribution visualization


In [ ]:
sns.histplot(x, kde=True, stat='density')  # histogram with KDE overlay for x


In [ ]:
plt.show()  # render histogram


# Box/violin plots across groups


In [ ]:
sns.boxplot(x='group', y='x', data=df)  # boxplots of x by group


In [ ]:
plt.show()  # render boxplot


In [ ]:
sns.violinplot(x='group', y='x', data=df, inner='quartile')  # violin plots (density) with quartiles


In [ ]:
plt.show()  # render violin plot


# Chi-square residuals visualization (standardized residual heatmap)


In [ ]:
chi2, p, dof, exp = stats.chi2_contingency(ct)  # expected counts from chi-square test


In [ ]:
std_resid = (ct - exp) / np.sqrt(exp)  # standardized residuals highlight cells driving association


In [ ]:
sns.heatmap(std_resid, annot=True, cmap='coolwarm', center=0)  # heatmap of residuals


In [ ]:
plt.show()  # render heatmap


# Probit/logit link functions (transform probabilities to linear predictors)


In [ ]:
stats.norm.ppf(0.8)  # probit: z such that Phi(z)=0.8 (inverse normal CDF)


In [ ]:
np.log(0.8 / (1 - 0.8))  # logit: log(p/(1-p)) for p=0.8


# Bayesian beta-binomial quick posterior (conjugate update demonstration)


In [ ]:
alpha_prior, beta_prior = 1, 1  # uniform Beta(1,1) prior for Bernoulli p


In [ ]:
successes = int(np.sum(binary)); trials = len(binary)  # observed data counts


In [ ]:
post_alpha, post_beta = alpha_prior + successes, beta_prior + (trials - successes)  # posterior Beta params


In [ ]:
from scipy.stats import beta as beta_dist  # Beta distribution for posterior summaries


In [ ]:
beta_dist.ppf([0.025, 0.975], a=post_alpha, b=post_beta)  # 95% Bayesian credible interval for p


# Rank-based effect size (Cliff’s delta) for two independent samples


In [ ]:
def cliffs_delta(a, b):
    # compute Cliff's delta: P(a>b) - P(a<b)
    a = np.asarray(a); b = np.asarray(b)
    count = 0
    for ai in a:
        count += np.sum(ai > b) - np.sum(ai < b)
    return count / (len(a) * len(b))


In [ ]:
cliffs_delta(x[cat == 'A'], x[cat == 'B'])  # effect size between groups A and B


# Proportion difference test (two-proportion z-test)


In [ ]:
count = np.array([df.loc[df.group == 'A', 'clicked'].sum(), df.loc[df.group == 'B', 'clicked'].sum()])  # successes per group


In [ ]:
nobs = np.array([np.sum(df.group == 'A'), np.sum(df.group == 'B')])  # trials per group


In [ ]:
sm_prop.proportions_ztest(count=count, nobs=nobs, alternative='two-sided')  # H0: p_A = p_B


# ANOVA effect size (eta-squared) from one-way ANOVA


In [ ]:
f_stat, p_val = stats.f_oneway(x[cat == 'A'], x[cat == 'B'], x[cat == 'C'])  # obtain F statistic


In [ ]:
ss_between = sum([len(g) * (np.mean(g) - np.mean(x))**2 for g in [x[cat == 'A'], x[cat == 'B'], x[cat == 'C']]])  # between-group SS


In [ ]:
ss_total = np.sum((x - np.mean(x))**2)  # total sum of squares


In [ ]:
eta_squared = ss_between / ss_total  # proportion of variance explained by group


In [ ]:
eta_squared  # report eta^2


# Empirical cumulative distribution function (ECDF)


In [ ]:
def ecdf(values):
    v = np.sort(values)
    p = np.arange(1, len(v) + 1) / len(v)
    return v, p


In [ ]:
vx, px = ecdf(x); (vx[:5], px[:5])  # first 5 ECDF points as example


# Kernel density estimate (KDE) quick evaluation


In [ ]:
kde = stats.gaussian_kde(x)  # fit Gaussian KDE to sample x


In [ ]:
kde.evaluate([0, 0.5, 1.0])  # estimate density at selected points


# Winsorization (limit extreme values)


In [ ]:
from scipy.stats.mstats import winsorize  # masked stats: winsorize replaces extreme tails


In [ ]:
winsorize(x, limits=[0.05, 0.05]).mean()  # 5% winsorized mean


# Jackknife resampling estimate for the mean’s bias


In [ ]:
def jackknife_mean(values):
    values = np.asarray(values)
    n = len(values)
    means = np.array([(np.sum(values) - values[i]) / (n - 1) for i in range(n)])
    return means.mean(), (n - 1) * (means.mean() - values.mean())  # (jackknife mean, bias estimate)


In [ ]:
jackknife_mean(x)  # returns (jackknife estimate, bias)


# Permutation test for difference in means (simple demonstration)


In [ ]:
def perm_test_mean(a, b, n_perm=2000, random_state=0):
    rng = np.random.default_rng(random_state)
    obs = np.mean(a) - np.mean(b)
    combined = np.concatenate([a, b])
    count = 0
    for _ in range(n_perm):
        rng.shuffle(combined)
        a_perm = combined[:len(a)]
        b_perm = combined[len(a):]
        count += (np.abs(np.mean(a_perm) - np.mean(b_perm)) >= np.abs(obs))
    return count / n_perm


In [ ]:
perm_test_mean(x, y, n_perm=1000, random_state=1)  # approximate two-sided p-value


# Bootstrap confidence interval for mean (percentile method)


In [ ]:
def bootstrap_ci_mean(values, n_boot=2000, alpha=0.05, random_state=0):
    rng = np.random.default_rng(random_state)
    means = np.empty(n_boot)
    n = len(values)
    for i in range(n_boot):
        sample = values[rng.integers(0, n, size=n)]
        means[i] = np.mean(sample)
    lower = np.percentile(means, 100 * (alpha / 2))
    upper = np.percentile(means, 100 * (1 - alpha / 2))
    return lower, upper


In [ ]:
bootstrap_ci_mean(x, n_boot=1000, alpha=0.05, random_state=2)  # 95% bootstrap CI for mean


# Correlation test with p-value (Pearson)


In [ ]:
stats.pearsonr(x, y)  # returns (correlation coefficient, two-sided p-value)


# Spearman correlation test with p-value


In [ ]:
stats.spearmanr(x, y)  # returns (rho, p-value)


# Kendall tau correlation test with p-value


In [ ]:
stats.kendalltau(x, y)  # returns (tau, p-value)


# Partial correlation (control for one variable) using residual approach


In [ ]:
def partial_corr(x, y, z):
    # regress x~z and y~z, then correlate residuals
    Z = sm.add_constant(z)
    res_x = sm.OLS(x, Z).fit().resid
    res_y = sm.OLS(y, Z).fit().resid
    return stats.pearsonr(res_x, res_y)


In [ ]:
partial_corr(df['x'].values, df['y'].values, (df['x'] + df['y']).values)  # toy example with z=x+y
